In [ ]:
###################
# LIBRARY IMPORTS#
###################
import twitter
import json
import os
import collections
import re
from tabulate import tabulate
import tweepy
import pandas as pd
from functools import partial
from sys import maxsize
import csv
import sys
import time
from urllib.error import URLError
from http.client import BadStatusLine
from textblob import TextBlob

In [1]:
###################
# CUSTOM FUNCTIONS#
###################

#########################################################################
#TAKES RAW TWITTER DATA AND PRINTS JSON#
##################  START   #######################################################
def pp(invar):
    f= json.dumps(invar, indent=1)
    print(f)
##################   END    ##########################################


#######################################################################
#CONNECT TO TWITTER API####
##################  START   ############################################
def oauth():
    CONSUMER_KEY = redacted
    CONSUMER_SECRET =redacted
    OAUTH_TOKEN = redacted
    OAUTH_TOKEN_SECRET = redacted

    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                           CONSUMER_KEY, CONSUMER_SECRET)

    twitter_api = twitter.Twitter(auth=auth)
    return twitter_api
####################  END  ##################################################

#####################################################################
#KEYWORD SEARCH
#q=keyword
#count
##################  START   ############################################
def twitter_search(q, count, twitter_api, **kw):
    
    search_results = twitter_api.search.tweets(q=q, count=count)
    statuses = search_results['statuses']

    for _ in range(10):
        try:
            next_results = search_results['search_metadata']['next_results']
        except KeyError: # No more results when next_results doesn't exist
            break

        kwargs = dict([ kv.split('=') 
            for kv in next_results[1:].split("&") ])
    
        search_results = twitter_api.search.tweets(**kwargs)
        statuses += search_results['statuses']
    return statuses
###################  END  ##################################################

#####################################################################
#EXPORT JSON FILE
#invar=variable containing data to be converted to json
#filename=file name for the json data to be save to
##################  START   ############################################
def json_export(invar, filename):
    data = json.dumps(invar, indent=1)
    file = open(filename, 'w')
    file.write(data)
    file.close()
###################  END  ##################################################

#####################################################################
#AGGREGATE RESULTS FROM ALL TXT FILES IN RAW TWEETS FOLDER
#path=path for the raw tweets
##################  START   ############################################
def agg_results():
    cwd = os.getcwd()
    path = cwd + "\\rawtweets" # change \\ to / if running on Mac computer
    files = os.listdir(path)
    agg_raw_data = []
    
    for infile in files:
        print(infile)
        filename = path + "\\" + infile # change \\ to / if running on Mac computer
        print(filename)
        with open(filename) as raw_data:
            raw_data = json.load(raw_data)
        agg_raw_data += raw_data
    return(agg_raw_data)
###################  END  ##################################################

In [ ]:
#####################################################################
################        MAIN FUNCTIOn   ############################

# twitter_api = oauth()
# search = ['bitcoin','XRP','etherium','litecoin','namecoin']
# result = []
# filename = 'Export.txt'
# cwd = os.getcwd()
# path = cwd
# for i in search:
#        result += twitter_search(i, 1000, twitter_api)
# json_export(result, filename)

results = agg_results()
json_export(results, "all_tweets.txt")

In [2]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

# converts .txt file into json structure
file = "all_tweets.txt"
with open(file) as raw_data:
    parsed_data = json.load(raw_data)

# finds indices of repeated tweets
texts = []
repeated_indices = []
for index in range(len(parsed_data)):
    text = parsed_data[index]['text']
    if text in texts:
        repeated_indices.append(index)
    else:
        texts.append(text)

# deletes repeated tweets
for index in sorted(repeated_indices, reverse = True): 
    del parsed_data[index]

8527


In [8]:
def top30words(json_tweet_data):
    # gathers the text from each tweet into a list, all converted to lowercase
    all_tweets = []
    for tweet in json_tweet_data:
        all_tweets.append(tweet['text'].lower())
    
    # counts the number of words in each tweet's text
    bag_of_words = [collections.Counter(re.findall(r'\w+', text)) for text in all_tweets]
    # collects all words and word counts together
    bags_sum = sum(bag_of_words, collections.Counter())
    
    # loads English stopwords and removes them from bag of words
    with open("stopwords/english") as file:
        stopword_list = file.readlines()
    stopword_list = [newline.strip() for newline in stopword_list] 
    for stopword in stopword_list:
        if stopword in bags_sum:
            del bags_sum[stopword]
            
    # prints table of top 30 words and their counts
    print(tabulate(bags_sum.most_common(30), headers=['Top Words', 'Count']))
    print("\n")

top30words(parsed_data)

Top Words         Count
--------------  -------
https              5173
co                 5017
bitcoin            3375
xrp                2690
namecoin           2372
btc                2278
rt                 2183
1                  2101
usd                1943
0                  1851
litecoin           1816
eth                1645
ltc                1442
cryptocurrency     1412
ripple             1147
etherium           1051
30                  880
mins                809
nmc                 787
ethereum            735
etc                 700
price               699
dash                634
neo                 629
crypto              572
blockchain          553
market              491
changed             443
omg                 434
bch                 412




In [10]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

def top10retweeted(json_tweet_data):
    # gathers number of retweet counts for each tweet into a list
    retweet_counts = []
    for tweet in json_tweet_data:
        retweet_counts.append(tweet['retweet_count'])

    # finds indices of top 10 retweet counts, ordered from most to least
    top10indices = sorted(range(len(retweet_counts)), key = lambda i: retweet_counts[i])[:-11:-1]
    
    # prints table of top 10 tweets and their retweet counts
    print("\033[4mCount\tTop Tweets\033[0m\n")
    for index in top10indices:
        print(str(json_tweet_data[index]['retweet_count']) + "\t\"" + json_tweet_data[index]['text'] + "\"")
    print("\n")

top10retweeted(parsed_data)

Count	Top Tweets

34169	"RT @lorde: VOGUE BABY AHHH! ✨🍌☺️ https://t.co/pxHDeyeFhd"
28713	"RT @stem910: 先日頭のいい友人に「喋り方がバカっぽいのをなんとかしたい」と相談したら「バカっぽいというのは本当はバカではないのにバカと思われてしまうということだが、お前は事実バカだろ」と指摘されたので「なる程。ではバカを強調する喋り方をやめたい」と相談し直したら“あ…"
14256	"RT @tsumland_jp: ✨最大100万円分の海外ディズニー旅行が当たるかも✨
ディズニー ツムツムランドの事前登録キャンペーンを実施中！
このアカウントをフォローするだけで豪華賞品が当たるチャンス🎵
この機会をお見逃しなく☆ ＃ツムツムランド
詳細↓
https://…"
7079	"RT @aco220: 友達が「彼氏と別れそう」って言うから、何事だ何があったのかどうしたのかって聞いたら、「Excelで下にずらして数字が並ぶの知らない人とは付き合えない」って、オートフィルわかんないとフラれるのやばいな"
5978	"RT @emartineeez: RT THIS TWEET IF YOU WANT TWO VIDEOS TODAY😁"
5651	"RT @at_raku: 東京ドームシティ アトラクションズでは、9/30（土）~「活撃 刀剣乱舞」とのコラボイベント『出陣！活撃 刀剣乱舞 in 東京ドームシティ』を開催！限定クリアファイルが貰えるスタンプラリー、コラボフード&amp;グッズも販売！https://t.co/KDW2…"
5604	"RT @ErikVoorhees: My memory is failing, was it Bitcoin or was it JP Morgan that was bailed out by the government? https://t.co/DHqFzr5UJN"
3828	"RT @huga731: #btc #bitcoin #xem #comsa #nem 100 000 уже зарегистрировалось в Японском ico COMSA .Это будет самое масштабное айсио

**3. Find the most popular Tweet Entities in your collection of tweets**

Please plot a table of the top 10 hashtags, top 10 user mentions that are the most popular in your collection of tweets.

In [11]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

def top10hashtags_user_mentions(json_tweet_data):
    user_list = []
    hashtag_list = []
    
    for tweet in json_tweet_data:
        # gathers the hashtags from each tweet into a list
        hashtags = tweet['entities']['hashtags']
        if len(hashtags):
            for hashtag in hashtags:
                hashtag_list.append("#" + hashtag['text'])
        
        # gathers the users mentioned in each tweet into a list
        user_mentions = tweet['entities']['user_mentions']
        if len(user_mentions):
            for user in user_mentions:
                user_list.append("@" + user['screen_name'])

    # counts number of each hashtag
    hashtag_count = collections.Counter(hashtag_list)
    # prints table of top 10 hashtags with their counts
    print(tabulate(hashtag_count.most_common(10), headers=['Top Hashtags', 'Count']))
    print("\n")
    
    # counts number of times each user is mentioned
    user_count = collections.Counter(user_list)
    # prints table of top 10 users mentioned and their counts
    print(tabulate(user_count.most_common(10), headers=['Top Users Mentioned', 'Count']))
    
top10hashtags_user_mentions(parsed_data)

Top Hashtags       Count
---------------  -------
#bitcoin            1188
#cryptocurrency     1180
#namecoin            737
#nmc                 708
#Bitcoin             687
#litecoin            567
#etherium            475
#Litecoin            421
#XRP                 376
#xrp                 352


Top Users Mentioned      Count
---------------------  -------
@Ripple                    125
@Keiki_XRP                 116
@cryptopayments2            71
@erishiiiii                 71
@SatoshiLite                69
@TO30447473                 68
@toyokichimaru              66
@cannavinothc               59
@MarketNmc                  51
@cryptographicc             49


In [13]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

CONSUMER_KEY = redacted
CONSUMER_SECRET =redacted
OAUTH_TOKEN = redacted
OAUTH_TOKEN_SECRET = redacted

auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                       CONSUMER_KEY, CONSUMER_SECRET)
twitter_api = twitter.Twitter(auth=auth)
#Start: Creating authorization for tweepy
auth1 = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)

auth1.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)


tweepy_api=tweepy.API(auth1,wait_on_rate_limit=True)
# End: Creating authorization for tweepy

#Start : Method to plot ID & Screen_name of 20 friends of any user
def plot_user_friends(popular_user) :
    friend_ids=[]
    friend_names=[]
    user = tweepy_api.get_user(popular_user)
    friend_list = user.friends()
    for friend in friend_list :
        friend_ids.append(friend.id_str)
        friend_names.append(friend.screen_name)
    friends_df = pd.DataFrame({"Friends ID": friend_ids, "Friends Screen name":friend_names })
    print(friends_df)
    
#End : Method to plot ID & Screen_name of 20 friends of any user

#Start : Method to plot ID & Screen_name of 20 followers of any user

def plot_user_followers(popular_user) :
    followers_ids=[]
    followers_names=[]
    user = tweepy_api.get_user(popular_user)
    followers_list = user.followers()
    for follower in followers_list :
        followers_ids.append(follower.id_str)
        followers_names.append(follower.screen_name)
    followers_df = pd.DataFrame({"Followers ID": followers_ids, "Followers Screen name":followers_names })
    print(followers_df)   
    
#End : Method to plot ID & Screen_name of 20 followers of any user

plot_user_friends('cryptocointalk')
print("\n")
plot_user_followers('cryptocointalk')

            Friends ID Friends Screen name
0   812491802955644928      EmbermineDrake
1   797930343277989888        TheEmbermine
2            104259801         Squidoogeek
3            262755165     SkinnerLiber8ed
4   788886561731538944              YVerif
5   902772556670914560          VUnioninfo
6           1479248557        msjemmagreen
7           4341007829           uquidcard
8   897380780510502912       CryptoTickets
9           2309343738     OfficialTitcoin
10  888615399281102848     GoldenFleece_co
11          2600194316         NewKoreCoin
12          1664012648       chimaera_tech
13          2313671966         NEMofficial
14           221071894              drbitq
15  867574261195522048           matryx_ai
16  873598891723239424          BlockDevCo
17  769176690908069888       godzillion_io
18  775658150179508224              iEx_ec
19  857130360903274500         Bitcore_BTX


          Followers ID Followers Screen name
0   910877157651394560           AjayBan7014
1    

In [1]:
import twitter
import tweepy
import time
import tweepy.error
from requests.exceptions import Timeout, ConnectionError
from requests.packages.urllib3.exceptions import ReadTimeoutError
import pandas as pd

# Creating variables to be used throughout the code
friends_list=[] 
friends_ids=[]
followers_list=[]
followers_ids=[]

# Start : Set authorisation for connection to Twitter & Tweepy api

CONSUMER_KEY = redacted
CONSUMER_SECRET =redacted
OAUTH_TOKEN = redacted
OAUTH_TOKEN_SECRET = redacted

auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                       CONSUMER_KEY, CONSUMER_SECRET)
auth1 = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)

auth1.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)

api=tweepy.API(auth1,wait_on_rate_limit=True)

# End : Set authorisation for connection to Twitter & Tweepy api

# Start : Function to get list of all friends of a user i.e. 'cryptocointalk' & store it in a text file
def get_friends():
    i=0
    j=0
    f = open('Friends_list.txt', 'w')
    users=tweepy.Cursor(api.friends, screen_name="cryptocointalk",count=200).items()
    while True:
            try:
                j=j+1
                user=next(users)
                i=i+1
                friends_list.append(user.screen_name)
                friends_ids.append(user.id)
                f.write(user.screen_name+'\n')
                if j>2900:
                    j=0
                    time.sleep(15*60)
            except (tweepy.TweepError) as ex:
                time.sleep(60*16)
                pass
            except (StopIteration):
                break


# End : Function to get list of all friends of a user i.e. 'cryptocointalk' & store it in a text file   

# Start : Function to get list of all followers of a user i.e. 'cryptocointalk' & store it in a text file
def get_followers():
    i=0
    j=0
    f = open('Followers_list.txt', 'w')
    users=tweepy.Cursor(api.followers, screen_name="cryptocointalk",count=200).items()
    while True:
            try:
                j=j+1
                user=next(users)
                i=i+1
                followers_list.append(user.screen_name)
                followers_ids.append(user.id)
                f.write(user.screen_name+'\n')
                if j>2900:
                    break
                    j=0
                    time.sleep(15*60)
            except (tweepy.TweepError) as ex:
                time.sleep(60*16)
                pass
            except (StopIteration):
                break

In [3]:
# Code to fetch list of mutual friends

CONSUMER_KEY = redacted
CONSUMER_SECRET =redacted
OAUTH_TOKEN = redacted
OAUTH_TOKEN_SECRET = redacted

auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                           CONSUMER_KEY, CONSUMER_SECRET)

twitter_api = twitter.Twitter(auth=auth)

auth1 = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)

auth1.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)

api=tweepy.API(auth1,wait_on_rate_limit=True)

print(twitter_api)



def make_twitter_request(twitter_api_func, max_errors=10, *args, **kw): 
    
    # A nested helper function that handles common HTTPErrors. Return an updated
    # value for wait_period if the problem is a 500 level error. Block until the
    # rate limit is reset if it's a rate limiting issue (429 error). Returns None
    # for 401 and 404 errors, which requires special handling by the caller.
    def handle_twitter_http_error(e, wait_period=2, sleep_when_rate_limited=True):
    
        if wait_period > 3600: # Seconds
            print('Too many retries. Quitting.', file=sys.stderr)
            raise e
    
        # See https://dev.twitter.com/docs/error-codes-responses for common codes
    
        if e.e.code == 401:
            print('Encountered 401 Error (Not Authorized)', file=sys.stderr)
            return None
        elif e.e.code == 404:
            print('Encountered 404 Error (Not Found)', file=sys.stderr)
            return None
        elif e.e.code == 429: 
            print('Encountered 429 Error (Rate Limit Exceeded)', file=sys.stderr)
            if sleep_when_rate_limited:
                print("Retrying in 15 minutes...ZzZ...", file=sys.stderr)
                sys.stderr.flush()
                time.sleep(60*15 + 5)
                print('...ZzZ...Awake now and trying again.', file=sys.stderr)
                return 2
            else:
                raise e # Caller must handle the rate limiting issue
        elif e.e.code in (500, 502, 503, 504):
            print('Encountered %i Error. Retrying in %i seconds' % \
                (e.e.code, wait_period), file=sys.stderr)
            time.sleep(wait_period)
            wait_period *= 1.5
            return wait_period
        else:
            raise e

    # End of nested helper function
    
    wait_period = 2 
    error_count = 0 

    while True:
        try:
            return twitter_api_func(*args, **kw)
        except twitter.api.TwitterHTTPError as e:
            error_count = 0 
            wait_period = handle_twitter_http_error(e, wait_period)
            if wait_period is None:
                return
        except URLError as e:
            error_count += 1
            time.sleep(wait_period)
            wait_period *= 1.5
            print("URLError encountered. Continuing.", file=sys.stderr)
            if error_count > max_errors:
                print("Too many consecutive errors...bailing out.", file=sys.stderr)
                raise
        except BadStatusLine as e:
            error_count += 1
            time.sleep(wait_period)
            wait_period *= 1.5
            print("BadStatusLine encountered. Continuing.", file=sys.stderr)
            if error_count > max_errors:
                print("Too many consecutive errors...bailing out.", file=sys.stderr)
                raise
                
def get_friends_followers_ids(twitter_api, screen_name=None, user_id=None,
                              friends_limit=maxsize, followers_limit=maxsize):
    assert (screen_name != None) != (user_id != None)
    get_friends_ids = partial(make_twitter_request, twitter_api.friends.ids, 
                              count=5000)
    get_followers_ids = partial(make_twitter_request, twitter_api.followers.ids, 
                                count=5000)
    friends_ids, followers_ids = [], []
    
    for twitter_api_func, limit, ids, label in [
                    [get_friends_ids, friends_limit, friends_ids, "friends"], 
                    [get_followers_ids, followers_limit, followers_ids, "followers"]
                ]:
        
        if limit == 0: continue
        
        cursor = -1
        while cursor != 0:
        
            # Use make_twitter_request via the partially bound callable...
            if screen_name: 
                response = twitter_api_func(screen_name=screen_name, cursor=cursor)
            else: # user_id
                response = twitter_api_func(user_id=user_id, cursor=cursor)

            if response is not None:
                ids += response['ids']
                cursor = response['next_cursor']
        
            print('Fetched {0} total {1} ids for {2}'.format(len(ids), 
                                                    label, (user_id or screen_name)), file=sys.stderr)
        
            # XXX: You may want to store data during each iteration to provide an 
            # an additional layer of protection from exceptional circumstances
        
            if len(ids) >= limit or response is None:
                break

    return friends_ids[:friends_limit], followers_ids[:followers_limit]


friends_ids, followers_ids = get_friends_followers_ids(twitter_api, 
                                                       screen_name="cryptocointalk", 
                                                       friends_limit=100000000000000000000000, 
                                                       followers_limit=100000000000000000000000)



friends_ids, followers_ids = set(friends_ids), set(followers_ids)
print('{0} has {1} mutual friends'.format('cryptocointalk', len(friends_ids.intersection(followers_ids))))

# get common followers & friends ids & store it in a list called result
result = []
for element in friends_ids:
    if element in followers_ids:
        result.append(element)


# Store mutual friends & followers in a dataframe & display it

mutual_friends_id = []
mutual_friends_names = []
for mutual in result :
    mutual_friends_id.append(mutual)
    mutual_friends_names.append(api.get_user(mutual).screen_name)
    
mutual_df = pd.DataFrame({"Mutual ID": mutual_friends_id, "Mutual Screen name":mutual_friends_names })
print(mutual_df)

Fetched 1000 total friends ids for cryptocointalk
Fetched 5000 total followers ids for cryptocointalk
Fetched 10000 total followers ids for cryptocointalk
Fetched 15000 total followers ids for cryptocointalk
Fetched 20000 total followers ids for cryptocointalk
Fetched 25000 total followers ids for cryptocointalk
Fetched 29731 total followers ids for cryptocointalk


cryptocointalk has 996 mutual friends
              Mutual ID Mutual Screen name
0    812491802955644928     EmbermineDrake
1    797930343277989888       TheEmbermine
2    788886561731538944             YVerif
3    902772556670914560         VUnioninfo
4    897380780510502912      CryptoTickets
5    888615399281102848    GoldenFleece_co
6    867574261195522048          matryx_ai
7    873598891723239424         BlockDevCo
8    769176690908069888      godzillion_io
9    775658150179508224             iEx_ec
10   857130360903274500        Bitcore_BTX
11   874241762075832322       mindpass2050
12   848547538957328387     bonzocorleonee
13   867498616730025985          HonestisN
14   867456902627721218     QchainPlatform
15   824342681912561678    MARXCOLLECTIVE1
16   862041575081246723       CONTENT_COIN
17            274456588      cryptocoinfan
18           2335207442          coinshost
19           3662979086        customminer
20             43192340           JackPhan
21           215

In [6]:
def agg_results(path):
    cwd = os.getcwd()
    path = cwd + "\\rawtweets" # change \\ to / if running on Mac computer
    print(path)
    files = os.listdir(path)
    results = []
    for infile in files:
        all_tweets = []
        filename=path + "\\" + infile # change \\ to / if running on Mac computer
        with open(filename) as raw_data:
            parsed_data = json.load(raw_data)
            for tweet in parsed_data:
                all_tweets.append(tweet['text'])
            result = sentiment_cat(all_tweets,infile)
            results.append(result)
            print(result)
    print_results(results, cwd + "\\output_problem4.csv") # change \\ to / if running on Mac computer
    return(results)

#####################################################################
#SENTIMENT ANALYSIS
#invar=variable containing data to be converted to json
#filename=file name for the json data to be save to
##################  START   ############################################
def sentiment_cat(all_tweets,infile):
    cntnodup = 0
    poscnt = 0
    pos_polar = 0
    pos_subj = 0
    negcnt = 0
    neg_polar = 0 
    neg_subj = 0 
    neutcnt = 0 
    neut_subj = 0
    result = []
    
    cntnodup += 1
    
    for i in range(0, len(all_tweets) - 1):
        cntnodup += 1
        x = TextBlob(all_tweets[i])
        if x.sentiment.polarity > 0:
            poscnt += 1
            pos_polar += x.sentiment.polarity
            pos_subj += x.sentiment.subjectivity

        elif x.sentiment.polarity == 0:
            neutcnt += 1
            neut_subj += x.sentiment.subjectivity

        else:
            negcnt += 1
            neg_polar += x.sentiment.polarity 
            neg_subj += x.sentiment.subjectivity
        
    result.append(infile)
    result.append(cntnodup)  
    result.append(poscnt)
    result.append(pos_polar / poscnt)  
    result.append(pos_subj / poscnt)  
    result.append(negcnt)  
    result.append(neg_polar / negcnt)  
    result.append(neg_subj / negcnt)  
    result.append(neutcnt)  
    result.append(neut_subj / negcnt)  
    return(result)

###################  END  ##################################################
def print_results(output, filename):
    with open(filename, "w") as f:
        writer = csv.writer(f)
        writer.writerows(output)



#####################################################################
################        MAIN FUNCTIOn   ############################   
cwd = os.getcwd()
path = cwd

agg_results(path)

/Users/renswny/Code/GitHub/DS501/rawtweets
['results_091317_12PM.txt', 1968, 576, 0.507860273587814, 0.4269610414662494, 103, -0.30785510269126803, 0.5390088296277613, 1288, 0.3863430420711973]
['results_091417_12PM.txt', 2040, 415, 0.3762832602053242, 0.5116602296632412, 93, -0.2770647803510707, 0.5007185690653437, 1531, 0.42889784946236537]
['results_091417_19.txt', 1688, 289, 0.3621229306395019, 0.4949375177878632, 107, -0.2850366972774976, 0.5403104186398582, 1291, 0.3822541165999109]
['results_091417_7PM.txt', 1688, 289, 0.3621229306395019, 0.4949375177878632, 107, -0.2850366972774976, 0.5403104186398582, 1291, 0.3822541165999109]
['results_091417_9AM.txt', 3162, 695, 0.3439175176633585, 0.4682033014281221, 176, -0.24354205304999876, 0.46386171225659845, 2290, 0.3450487012987012]
['results_091517_12PM.txt', 2917, 601, 0.3975863140381918, 0.5278367373521283, 207, -0.270254488599805, 0.4756835839504928, 2108, 0.2559983896940417]
['results_091517_7PM.txt', 1560, 459, 0.31498889651259

[['results_091317_12PM.txt',
  1968,
  576,
  0.507860273587814,
  0.4269610414662494,
  103,
  -0.30785510269126803,
  0.5390088296277613,
  1288,
  0.3863430420711973],
 ['results_091417_12PM.txt',
  2040,
  415,
  0.3762832602053242,
  0.5116602296632412,
  93,
  -0.2770647803510707,
  0.5007185690653437,
  1531,
  0.42889784946236537],
 ['results_091417_19.txt',
  1688,
  289,
  0.3621229306395019,
  0.4949375177878632,
  107,
  -0.2850366972774976,
  0.5403104186398582,
  1291,
  0.3822541165999109],
 ['results_091417_7PM.txt',
  1688,
  289,
  0.3621229306395019,
  0.4949375177878632,
  107,
  -0.2850366972774976,
  0.5403104186398582,
  1291,
  0.3822541165999109],
 ['results_091417_9AM.txt',
  3162,
  695,
  0.3439175176633585,
  0.4682033014281221,
  176,
  -0.24354205304999876,
  0.46386171225659845,
  2290,
  0.3450487012987012],
 ['results_091517_12PM.txt',
  2917,
  601,
  0.3975863140381918,
  0.5278367373521283,
  207,
  -0.270254488599805,
  0.4756835839504928,
  2108,
